# Parameter Estimation with AdamW

In this notebook, we demonstrate an example of parameter estimation for a single-particle model using the AdamW optimiser [1][2]. The AdamW optimiser is an algorithm for gradient-based optimisation, combining the advantages of the Adaptive Gradient Algorithm (AdaGrad) and Root Mean Square Propagation (RMSProp).

[[1]: Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980) 

[[2]: Decoupled Weight Decay Regularization](https://doi.org/10.48550/arXiv.1711.05101)

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Generate a synthetic dataset

To demonstrate parameter estimation, we first need some data. We will generate synthetic data using a PyBaMM forward model, which requires defining parameter values and the model itself. We can then simulate the model using the default constant discharge current to generate voltage data.

In [ ]:
model = pybamm.lithium_ion.SPM()
parameter_values = pybamm.ParameterValues("Chen2020")

sim = pybamm.Simulation(model, parameter_values=parameter_values)
t_eval = np.arange(0, 900, 2)
solution = sim.solve(t_eval=t_eval)
current = solution["Current [A]"](t_eval)
voltage = solution["Voltage [V]"](t_eval)

To make the parameter estimation more realistic, we add Gaussian noise to the data. The dataset for optimisation is composed of time, current, and the noisy voltage data.

In [ ]:
sigma = 0.001  # 1 mV
corrupt_values = voltage + np.random.normal(0, sigma, len(t_eval))

dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": current,
        "Voltage [V]": corrupt_values,
    }
)

## Identifying the parameters

We select the model parameters for estimation and set up their prior distributions and bounds:

In [ ]:
parameter_values.update(
    {
        "Negative electrode active material volume fraction": pybop.Parameter(
            prior=pybop.Gaussian(0.6, 0.02),
            bounds=[0.5, 0.8],
        ),
        "Positive electrode active material volume fraction": pybop.Parameter(
            prior=pybop.Gaussian(0.48, 0.02),
            bounds=[0.4, 0.7],
        ),
    }
)

With the dataset and parameters defined, we can set up the optimisation problem with a cost function, and the optimiser.

In [ ]:
simulator = pybop.pybamm.Simulator(
    model, parameter_values=parameter_values, protocol=dataset
)
cost = pybop.SumSquaredError(dataset)
problem = pybop.Problem(simulator, cost)
optim = pybop.AdamW(problem)
optim.set_max_unchanged_iterations(40)
optim.set_max_iterations(150)

# Reduce the momentum influence for the reduced number of optimiser iterations
optim.optimiser.b1 = 0.75
optim.optimiser.b2 = 0.75

NOTE: Boundaries ignored by <class 'pybop.optimisers._adamw.AdamWImpl'>


We proceed to run the AdamW optimisation algorithm to estimate the parameters:

In [ ]:
result = optim.run()

After the optimisation, we can examine the estimated parameter values:

In [ ]:
result.best_inputs  # This will output the estimated parameters

{'Negative electrode active material volume fraction': 0.7503362663894575,
 'Positive electrode active material volume fraction': 0.6648529316910259}

## Plotting and visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation. We can plot the system's response using the estimated parameters compared to the target:

### Comparing system response

We can quickly plot the system's response using the estimated parameters compared to the target:

In [ ]:
pybop.plot.problem(problem, inputs=result.best_inputs, title="Optimised Comparison");

### Convergence and parameter trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
result.plot_convergence()
result.plot_parameters();

### Cost landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:

In [ ]:
# Plot the cost landscape with updated bounds
bounds = np.asarray([[0.6, 0.9], [0.5, 0.8]])
result.plot_surface(bounds=bounds);

## Concluding thoughts

This notebook illustrates how to perform parameter estimation using AdamW in PyBOP, providing insights into the optimisation process through various visualisations.